In [5]:
import os
from tqdm import tqdm
import pandas as pd
from concurrent.futures import ProcessPoolExecutor, as_completed
from itertools import combinations, islice
from tqdm import tqdm
import aliprocess
import importlib
importlib.reload(aliprocess)

<module 'aliprocess' from 'D:\\Documents\\PhD\\Code\\alibaba-explore\\aliprocess.py'>

In [2]:
wd = "//wsl.localhost/ubuntu/home/gw240/projects/alibaba/clusterdata/cluster-trace-microservices-v2022/data/CallGraph/"
# Path to the directory containing your files
directory_path = wd
files = os.listdir(directory_path)
filename = files[0]
file_path = os.path.join(directory_path, filename)
# Load the DataFrame
df = pd.read_csv(file_path, compression='gzip', header=0, sep=',', quotechar='"',on_bad_lines="warn");
MSCallGraph = df

# Create a unique identifier for each call pattern
MSCallGraph['call_pattern'] = list(zip(MSCallGraph['um'], MSCallGraph['dm']))

# Group by service and traceid, then create a list of call_pattern tuples for each group
grouped = MSCallGraph.groupby(['service', 'traceid'])['call_pattern'].apply(list).reset_index()

# Now group by service only, to create a list of lists of call patterns for each service
service_call_patterns = grouped.groupby('service')['call_pattern'].apply(list).reset_index(name='pattern_list')

Skipping line 3685: expected 11 fields, saw 12
Skipping line 4483: expected 11 fields, saw 12
Skipping line 9698: expected 11 fields, saw 12
Skipping line 9703: expected 11 fields, saw 12
Skipping line 16094: expected 11 fields, saw 12
Skipping line 19027: expected 11 fields, saw 12
Skipping line 19479: expected 11 fields, saw 12
Skipping line 19968: expected 11 fields, saw 12
Skipping line 21909: expected 11 fields, saw 12
Skipping line 21911: expected 11 fields, saw 12
Skipping line 32127: expected 11 fields, saw 12
Skipping line 32128: expected 11 fields, saw 12
Skipping line 32849: expected 11 fields, saw 12
Skipping line 33131: expected 11 fields, saw 12
Skipping line 33132: expected 11 fields, saw 12
Skipping line 33136: expected 11 fields, saw 12
Skipping line 33137: expected 11 fields, saw 12
Skipping line 34262: expected 11 fields, saw 12
Skipping line 34263: expected 11 fields, saw 12
Skipping line 34265: expected 11 fields, saw 12
Skipping line 34267: expected 11 fields, saw

In [9]:
# Dictionary to store the results
service_similarities = {}
# Submit tasks for parallel execution
for index, row in tqdm(service_call_patterns.iterrows(), total=len(service_call_patterns)):
    service = row['service']
    patterns = row['pattern_list']
    if 500 < len(patterns) < 10000:  # Only parallelize if more than 5000 patterns
        service_similarities[service] = aliprocess.mean_jaccard_similarity(patterns)
    elif len(patterns) > 20000:
        print(len(patterns))
        service_similarities[service] = aliprocess.mean_jaccard_similarity_parallel(patterns, 150000, num_workers=20)
    elif len(patterns) != 1:
        service_similarities[service] = (len(patterns), aliprocess.calculate_similarity_for_service(patterns))
    else:
        service_similarities[service] = (1, 1)

  3%|▎         | 3000/100186 [02:31<1:21:50, 19.79it/s]


KeyboardInterrupt: 

In [ ]:
# Dictionary to store the results
service_similarities = {}

# Set up the process pool for parallel execution
with ProcessPoolExecutor() as executor:
    futures = {}
    # Submit tasks for parallel execution
    for index, row in tqdm(service_call_patterns.iterrows(), total=len(service_call_patterns)):
        service = row['service']
        patterns = row['pattern_list']
        if 500 < len(patterns) < 10000:  # Only parallelize if more than 5000 patterns
            futures[executor.submit(aliprocess.calculate_similarity_for_service, patterns)] = service
        elif len(patterns) > 10000:
            print(len(patterns))
            service_similarities[service] = aliprocess.mean_jaccard_similarity_parallel(patterns, 150000, num_workers=20)
        elif len(patterns) != 1:
            service_similarities[service] = (len(patterns), aliprocess.calculate_similarity_for_service(patterns))
        else:
            service_similarities[service] = (1, 1)

    # Retrieve results as they are completed
    for future in tqdm(as_completed(futures), total=len(futures)):
        service = futures[future]
        try:
            similarity = future.result()
            service_similarities[service] = similarity
        except Exception as e:
            print(f"Service {service} generated an exception: {e}")

In [ ]:
large_patterns = {}
for index, row in tqdm(service_call_patterns.iterrows(), total=len(service_call_patterns)):
    service = row['service']
    patterns = row['pattern_list']
    if len(patterns) > 90000:
        large_patterns[service] = patterns
        break
        

In [ ]:
aliprocess.mean_jaccard_similarity_parallel(patterns, 150000, num_workers=20)

Creating batch for similarity


In [0]:
# def mean_jaccard_similarity(patterns):
#     # Generator expression - calculates one similarity at a time
#     similarities = (aliprocess.jaccard_similarity(set(a), set(b)) for a, b in combinations(patterns, 2))
#     total, count = 0, 0
#     n = (((len(patterns)**2)- len(patterns))/2) 
#     for similarity in tqdm(similarities,total=n):
#         total += similarity
#         count += 1
#     return total / count if count else 0
# 
# mean_jaccard_similarity(patterns)

In [7]:
aliprocess.mean_jaccard_similarity_parallel(patterns, 150000, num_workers=20)

Creating batch for similarity
Calculating batch similarity


Calculating Similarities: 100%|██████████| 2341/2341 [01:01<00:00, 37.91it/s] 


0.32286346742604016

In [8]:
len(patterns)

26499

In [0]:
# Dictionary to store the results
service_similarities = {}

# # Set up the process pool for parallel execution
# with ProcessPoolExecutor() as executor:
#     futures = {}
#     # Submit tasks for parallel execution
#     for index, row in tqdm(service_call_patterns.iterrows(), total=len(service_call_patterns)):
#         service = row['service']
#         patterns = row['pattern_list']
#         if 500 < len(patterns) < 10000:  # Only parallelize if more than 5000 patterns
#             futures[executor.submit(calculate_similarity_for_service, patterns)] = service
#         elif len(patterns) > 10000:
#             batch_size = 1000
#             service_similarities[service] = process_batches(patterns, batch_size)
#         elif len(patterns) != 1:
#             service_similarities[service] = (len(patterns), calculate_similarity_for_service(patterns))
#         else:
#             service_similarities[service] = (1, 1)
#
#     # Retrieve results as they are completed
#     for future in tqdm(as_completed(futures), total=len(futures)):
#         service = futures[future]
#         try:
#             similarity = future.result()
#             service_similarities[service] = similarity
#         except Exception as e:
#             print(f"Service {service} generated an exception: {e}")
#
# print("Aggregating into dataframe")
# # Convert results to DataFrame for better visualization and analysis
# df_results = pd.DataFrame.from_dict(service_similarities, orient='index', columns=['mean_jaccard_similarity'])
#
# # Display the results
# print(df_results)